# Distance Traveled Visualization Book

In [4]:
# Dependencies
import pandas as pd
import numpy as np
import os
import sys


path_to_table = os.path.join(os.getcwd(), '..', 'data', 'output', 'Team_Travel_Information_v2_09-25-2024.csv')
df = pd.read_csv(path_to_table)

# Get the list of teams
teams = df['Team'].unique()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Team                    64 non-null     object 
 1   Reg_Distance            64 non-null     float64
 2   Reg_Trips               64 non-null     int64  
 3   Reg_AVG                 64 non-null     float64
 4   N_Distance              64 non-null     float64
 5   Neutral_Site_Trips      64 non-null     int64  
 6   N_AVG                   64 non-null     float64
 7   Total_Distance          64 non-null     float64
 8   Overall_AVG             64 non-null     float64
 9   Closest_Team            64 non-null     object 
 10  Closest_Distance        64 non-null     float64
 11  Total_Closest_Matches   64 non-null     int64  
 12  Longest_Trip_Opponent   64 non-null     object 
 13  Distance_Longest_Trip   64 non-null     float64
 14  Game_Type_Longest_Trip  64 non-null     obje